In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numba import jit
import swifter
from multiprocessing import Pool

/home/igo/Mineracao de Dados/TP1/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
N = 100000
PATH = "microdados_enem_2022/DADOS/"

In [3]:
questions = pd.read_csv(PATH + "ITENS_PROVA_2022.csv",encoding="latin",sep=";")


In [4]:
prova_azul = questions[questions["TX_COR"] == "AZUL"]

In [5]:
hab_questionary = pd.read_csv(PATH + "QUEST_HAB_ESTUDO.csv",encoding="latin",sep=";", nrows=N)

In [6]:
questions_features = ["NU_INSCRICAO", "TX_RESPOSTAS_CN","TX_RESPOSTAS_CH","TX_RESPOSTAS_LC","TX_RESPOSTAS_MT",'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
       'CO_PROVA_MT']

values_to_match = {
    'CO_PROVA_CN': 1085,
    'CO_PROVA_CH': 1055,
    'CO_PROVA_LC': 1065,
    'CO_PROVA_MT': 1075,
}
answers_features = questions_features[1:]

In [7]:
microdados = pd.read_csv(PATH + "MICRODADOS_ENEM_2022.csv",encoding="latin",sep=";", nrows=N)

In [8]:
microdados = microdados[questions_features].dropna()

In [9]:
microdados_azul = microdados[(microdados['CO_PROVA_MT'] == values_to_match['CO_PROVA_MT'])]

In [10]:
gabarito_CN = prova_azul[prova_azul["CO_PROVA"] == values_to_match['CO_PROVA_CN']].sort_values(by='CO_POSICAO')["TX_GABARITO"]
gabarito_CH = prova_azul[prova_azul["CO_PROVA"] == values_to_match['CO_PROVA_CH']].sort_values(by='CO_POSICAO')["TX_GABARITO"]
gabarito_LC = prova_azul[(prova_azul["CO_PROVA"] == values_to_match['CO_PROVA_LC']) & (prova_azul["TP_LINGUA"] != 1)].sort_values(by='CO_POSICAO')["TX_GABARITO"]
gabarito_MT = prova_azul[prova_azul["CO_PROVA"] == values_to_match['CO_PROVA_MT']].sort_values(by='CO_POSICAO')["TX_GABARITO"]

gabaritos = [gabarito_CN, gabarito_CH, gabarito_LC, gabarito_MT]

In [11]:
respostas_MT = microdados_azul["TX_RESPOSTAS_MT"]

In [12]:
def vectorize_strings(string):
    return np.array(list(string))

In [13]:
vectorized_resps = respostas_MT.swifter.apply(vectorize_strings)

Pandas Apply: 100%|██████████| 17017/17017 [00:00<00:00, 78577.00it/s]


In [14]:
right_wrong_MT = vectorized_resps.swifter.apply(lambda x: x == gabarito_MT)

Pandas Apply: 100%|██████████| 17017/17017 [00:02<00:00, 6897.33it/s] 


In [15]:
right_wrong_MT.columns = range(1,46)

In [16]:
right_wrong_MT["NU_INCRICAO"] = microdados_azul["NU_INSCRICAO"]
right_wrong_MT

,1,2,3,4,5,6,7,8,9,10,...,37,38,39,40,41,42,43,44,45,NU_INCRICAO
31,False,False,False,True,False,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,210054534656
33,False,True,False,False,True,False,True,True,False,True,...,False,False,False,True,False,False,False,False,True,210057182714
36,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,210056185147
37,True,False,False,False,False,False,True,False,False,False,...,False,False,False,True,False,True,False,True,False,210056368452
38,False,False,True,False,True,False,False,True,False,False,...,False,False,True,False,True,False,False,False,True,210057293916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99982,False,True,True,False,True,False,False,True,True,False,...,True,True,False,True,False,True,False,True,False,210056346470
99983,False,True,False,False,False,False,False,True,True,False,...,False,True,False,False,True,False,False,False,False,210056678541
99994,False,False,False,True,True,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,210056387099
99996,False,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,210055141955


In [17]:
def task(feature):
        i,area = feature
        print(area)
        respostas = microdados[area]
        vectorized_resps   = respostas.swifter.apply(vectorize_strings)
        right_wrong_corr   = vectorized_resps.swifter.apply(lambda x: x == gabaritos[i])
        right_wrong_corr["NU_INCRICAO"] = microdados["NU_INSCRICAO"]
        right_wrong_corr.to_csv(PATH + area + ".csv",sep=";")

def get_gabaritos(answers_features,microdados,gabaritos):
    values = list(enumerate(answers_features))
    with Pool() as pool:
        pool.map(task,values)
get_gabaritos(answers_features[:-4],microdados_azul,gabaritos)

AttributeError: Can't pickle local object 'get_gabaritos.<locals>.task'

[(0, 'TX_RESPOSTAS_CN'),
 (1, 'TX_RESPOSTAS_CH'),
 (2, 'TX_RESPOSTAS_LC'),
 (3, 'TX_RESPOSTAS_MT'),
 (4, 'CO_PROVA_CN'),
 (5, 'CO_PROVA_CH'),
 (6, 'CO_PROVA_LC'),
 (7, 'CO_PROVA_MT')]